In [1]:
import datetime

from dagster.core.execution import create_execution_plan

from dagster.utils import load_yaml_from_path, script_relative_path

from dagster_airflow import scaffold_airflow_dag

from test_project.dagster_airflow_demo import define_demo_execution_pipeline


In [2]:
IMAGE = 'dagster-airflow-demo'


In [3]:
    pipeline = define_demo_execution_pipeline()
    env_config = load_yaml_from_path(script_relative_path('test_project/env.yml'))


In [4]:
create_execution_plan(pipeline, env_config).topological_steps()


[ExecutionStep(key='multiply_the_word.word.input_thunk', step_inputs=[], step_input_dict={}, step_outputs=[StepOutput(name='input_thunk_output', runtime_type=<dagster.core.types.runtime.String object at 0x10e343080>)], step_output_dict={'input_thunk_output': StepOutput(name='input_thunk_output', runtime_type=<dagster.core.types.runtime.String object at 0x10e343080>)}, compute_fn=<function _create_input_thunk_execution_step.<locals>._fn at 0x10e242d90>, kind=<StepKind.INPUT_THUNK: 'INPUT_THUNK'>, solid=<dagster.core.definitions.dependency.Solid object at 0x10e4d3860>, tags={'step_key': 'multiply_the_word.word.input_thunk', 'pipeline': 'demo_pipeline', 'solid': 'multiply_the_word', 'solid_definition': 'multiply_the_word', 'input': 'word'}),
 ExecutionStep(key='multiply_the_word.transform', step_inputs=[StepInput(name='word', runtime_type=<dagster.core.types.runtime.String object at 0x10e343080>, prev_output_handle=StepOutputHandle(step="multiply_the_word.word.input_thunk", output_name="i

In [5]:
import graphql

In [6]:
step = create_execution_plan(pipeline, env_config).topological_steps()[0]

In [7]:
step.step_outputs

[StepOutput(name='input_thunk_output', runtime_type=<dagster.core.types.runtime.String object at 0x10e343080>)]

In [8]:
step = create_execution_plan(pipeline, env_config).topological_steps()[1]

In [9]:
step.step_inputs[0]_output

SyntaxError: invalid syntax (<ipython-input-9-ca7f2b1c4913>, line 1)

In [10]:
from dagster_airflow.utils import IndentingBlockPrinter


In [11]:
def _normalize_key(key):
    '''We need to make sure task ids play nicely with Airflow.'''
    return key.replace('_', '__').replace('.', '_')


In [15]:
def _key_for_marshalled_result(step_key, result_name):
    return _normalize_key(step_key) + '___' + _normalize_key(result_name) + '.pickle'


def _make_marshalled_inputs_for_step(step):
    if not step.step_inputs:
        return '[]'

    printer = IndentingBlockPrinter(indent_level=2)
    printer.line('[')
    with printer.with_indent():
        for step_input in step.step_inputs:
            printer.line('{')
            with printer.with_indent():
                printer.line('inputName: "{input_name}",'.format(input_name=step_input.name))
                printer.line(
                    'key: "{key}"'.format(
                        key=_key_for_marshalled_result(
                            step_input.prev_output_handle.step.key,
                            step_input.prev_output_handle.output_name,
                        )
                    )
                )
            printer.line('}')
    printer.line(']')

    return printer.read()


def _make_marshalled_outputs_for_step(step):
    if not step.step_outputs:
        return '[]'

    printer = IndentingBlockPrinter(indent_level=2)
    printer.line('[')
    with printer.with_indent():
        for step_output in step.step_outputs:
            printer.line('{')
            with printer.with_indent():
                printer.line('outputName: "{output_name}",'.format(output_name=step_output.name))
                printer.line(
                    'key: "{key}"'.format(
                        key=_key_for_marshalled_result(step.key, step_output.name)
                    )
                )
            printer.line('}')
    printer.line(']')

    return printer.read()

In [16]:
_make_marshalled_inputs_for_step(step)

'[\n  {\n    inputName: "word",\n    key: "multiply__the__word_word_input__thunk___input__thunk__output.pickle"\n  }\n]\n'

In [17]:
_make_marshalled_outputs_for_step(step)

'[\n  {\n    outputName: "result",\n    key: "multiply__the__word_transform___result.pickle"\n  }\n]\n'

In [23]:
step

ExecutionStep(key='multiply_the_word.word.input_thunk', step_inputs=[], step_input_dict={}, step_outputs=[StepOutput(name='input_thunk_output', runtime_type=<dagster.core.types.runtime.String object at 0x10ab13a90>)], step_output_dict={'input_thunk_output': StepOutput(name='input_thunk_output', runtime_type=<dagster.core.types.runtime.String object at 0x10ab13a90>)}, compute_fn=<function _create_input_thunk_execution_step.<locals>._fn at 0x10ad24ea0>, kind=<StepKind.INPUT_THUNK: 'INPUT_THUNK'>, solid=<dagster.core.definitions.dependency.Solid object at 0x10ad2c780>, tags={'step_key': 'multiply_the_word.word.input_thunk', 'pipeline': 'demo_pipeline', 'solid': 'multiply_the_word', 'solid_definition': 'multiply_the_word', 'input': 'word'})

In [18]:
CONFIG = \
    {'context': {'default': {'config': {'log_level': 'DEBUG'}}},
     'solids': {'multiply_the_word': {'config': {'factor': 2},
                                      'inputs': {'word': {'value': 'bar'}}}}}


In [23]:
print(
'''-q '
            mutation {{
              startSubplanExecution(
                config: {config},
                executionMetadata: {{
                  runId: "testRun"
                }},
                pipelineName: "demo_pipeline",
                stepExecutions: [
                  {{
                    stepKey: "multiply_the_word.word.input_thunk"
                  }}
                ],
                marshalledInputs: 
                [
                ],
                marshalledOutputs: 
                [
                  {{
                    outputName: "input_thunk_output",
                    key: "multiply__the__word_word_input__thunk___input__thunk__output.pickle"
                  }}
                ],
              ) {{
                __typename
              }}
            '
        '''.format(config=CONFIG)
)

-q '
            mutation {
              startSubplanExecution(
                config: {'context': {'default': {'config': {'log_level': 'DEBUG'}}}, 'solids': {'multiply_the_word': {'config': {'factor': 2}, 'inputs': {'word': {'value': 'bar'}}}}},
                executionMetadata: {
                  runId: "testRun"
                },
                pipelineName: "demo_pipeline",
                stepExecutions: [
                  {
                    stepKey: "multiply_the_word.word.input_thunk"
                  }
                ],
                marshalledInputs: 
                [
                ],
                marshalledOutputs: 
                [
                  {
                    outputName: "input_thunk_output",
                    key: "multiply__the__word_word_input__thunk___input__thunk__output.pickle"
                  }
                ],
              ) {
                __typename
              }
            '
        


In [24]:
dir(graphql)

['BREAK',
 'DEFAULT_DEPRECATION_REASON',
 'DirectiveLocation',
 'GraphQLArgument',
 'GraphQLBackend',
 'GraphQLBoolean',
 'GraphQLCachedBackend',
 'GraphQLCoreBackend',
 'GraphQLDeciderBackend',
 'GraphQLDeprecatedDirective',
 'GraphQLDirective',
 'GraphQLDocument',
 'GraphQLEnumType',
 'GraphQLEnumValue',
 'GraphQLError',
 'GraphQLField',
 'GraphQLFloat',
 'GraphQLID',
 'GraphQLIncludeDirective',
 'GraphQLInputObjectField',
 'GraphQLInputObjectType',
 'GraphQLInt',
 'GraphQLInterfaceType',
 'GraphQLList',
 'GraphQLNonNull',
 'GraphQLObjectType',
 'GraphQLScalarType',
 'GraphQLSchema',
 'GraphQLSkipDirective',
 'GraphQLString',
 'GraphQLUnionType',
 'MiddlewareManager',
 'ParallelVisitor',
 'ResolveInfo',
 'SchemaMetaFieldDef',
 'Source',
 'TypeInfo',
 'TypeInfoVisitor',
 'TypeKind',
 'TypeMetaFieldDef',
 'TypeNameMetaFieldDef',
 'Undefined',
 'VERSION',
 '__Directive',
 '__DirectiveLocation',
 '__EnumValue',
 '__Field',
 '__InputValue',
 '__Schema',
 '__Type',
 '__TypeKind',
 '__all__

In [26]:
graphql.parse_value('{"context": {"default": {"config": {"log_level": "DEBUG"}}}, "solids": {"multiply_the_word": {"config": {"factor": 2}, "inputs": {"word": {"value": "bar"}}}}}')

GraphQLSyntaxError: Syntax Error GraphQL (1:2) Expected Name, found String "context"

1: {"context": {"default": {"config": {"log_level": "DEBUG"}}}, "solids": {"multiply_the_word": {"config": {"factor": 2}, "inputs": {"word": {"value": "bar"}}}}}
    ^
